# ПР №6

Выполнил Сердюков Матвей, ББМО-01-23

## Импорт библиотек

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod

## Загрузка датасета и преобразование данных

In [8]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

## Обучение модели

In [9]:
model = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)

Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8645 - loss: 0.4718 - val_accuracy: 0.9667 - val_loss: 0.1285
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9646 - loss: 0.1246 - val_accuracy: 0.9732 - val_loss: 0.0975
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9758 - loss: 0.0813 - val_accuracy: 0.9762 - val_loss: 0.0806
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9824 - loss: 0.0560 - val_accuracy: 0.9767 - val_loss: 0.0839
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9870 - loss: 0.0450 - val_accuracy: 0.9782 - val_loss: 0.0829


## Проведение атаки

In [10]:
classifier = TensorFlowV2Classifier(model=model, nb_classes=10, input_shape=(28, 28, 1), loss_object=tf.keras.losses.SparseCategoricalCrossentropy())

attack = FastGradientMethod(estimator=classifier, eps=0.1)

x_test_adv = attack.generate(x=x_test)

predictions = np.argmax(classifier.predict(x_test_adv), axis=1)
accuracy = np.sum(predictions == y_test) / len(y_test)
print(f'Accuracy on adversarial samples: {accuracy * 100:.2f}%')

Accuracy on adversarial samples: 2.87%
